# Language Model fine-tuning using MLM task

## 0. Define Hyperparameters

In [1]:
AVAILABLE_GPU = 2 # Available GPU with 0% usage
HUGGINGFACE_MODEL = "dccuchile/bert-base-spanish-wwm-uncased" # HuggingFace pre-trained model to train
MODEL_SAVE_PATH = "./output/old-spanish-beto-uncased.pt" # Path to save the trained model

MASK_PROB = 0.15 # Probability of masking within a text
LR = 2e-5 # Learning rate for Adam
EPOCHS = 5 # Number of epochs to train
BATCH_SIZE = 16 # Batch size for training
MAX_TOKENIZER_LENGTH = 512 # Maximum length of the texts in the tokenizer

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = f"{AVAILABLE_GPU}"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
tf_device=f'/gpu:{AVAILABLE_GPU}'

from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from torch.optim import Adam
import nltk
import pandas as pd
from datasets import Dataset
nltk.download('punkt')

/home/historynlp/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/historynlp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Load pre-trained LM for MLM and it's tokenizer

Define the tokenizer and the model, using the Model's `transformers` ID from Hugging Face. It's important to consider the possible differences between _cased_ and _uncased_ models.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL)
model = AutoModelForMaskedLM.from_pretrained(HUGGINGFACE_MODEL)

In [5]:
CLS_TOKEN = tokenizer.cls_token_id
SEP_TOKEN = tokenizer.sep_token_id
PAD_TOKEN = tokenizer.pad_token_id
MASK_TOKEN = tokenizer.mask_token_id
CLS_TOKEN, SEP_TOKEN, PAD_TOKEN, MASK_TOKEN

(4, 5, 1, 0)

## 2. Load and pre-process dataset

For training the model, we'll take the training split from the full spanish corpus, after cleaning. Given after the preparation stage:
> This corpus is already chunked for no more than 512 tokens, so there will be no chunking required for models that has this maximum length

In [ ]:
df = pd.read_csv("./data/old-spanish-corpus-chunked.tsv", sep="\t", usecols=["text"])
df = df[(df.source != "19th century Latam Newspapers")]
df.reset_index(drop=True, inplace=True)

dataset = Dataset.from_pandas(df)
dataset

A random sample of a text:

In [7]:
dataset[10000]["text"]

'--Tal vez se trate de una simple filtración--dijo Van-Horn--. Tenemos bomba a bordo y luego la haremos funcionar.'

## 3. Tokenize and Mask the dataset

Now, it's possible to start the retraining process of the model for the MLM task. For that, we'll first tokenize the input texts:

In [8]:
%%capture tokenizer_output
%%time

# All the texts of our dataset are stored in dataset["text"]
inputs = tokenizer(dataset["text"], return_tensors='pt', max_length=MAX_TOKENIZER_LENGTH, truncation=True, padding='max_length')
inputs

In [9]:
tokenizer_output.show()

CPU times: user 13min 59s, sys: 2min 47s, total: 16min 46s
Wall time: 4min


{'input_ids': tensor([[    4,  1098,  1000,  ...,     1,     1,     1],
        [    4,  1413,   998,  ...,     1,     1,     1],
        [    4,  1032,  2168,  ...,     1,     1,     1],
        ...,
        [    4,  1057,  9447,  ...,     1,     1,     1],
        [    4,  1032,  3876,  ...,     1,     1,     1],
        [    4,  1265, 30957,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Here's a comparation between the masked sentence and the original one:

In [10]:
dataset[100]['text']

'Comenzóseme a hacer áspera la morada y desapacibles los zaguanes. Fuí entrando poco a poco entre unos sastres que se me llegaron, que iban medrosos de los diablos. En la primera entrada hallamos siete demonios escribiendo los que íbamos entrando. Preguntáronme mi nombre; díjele y pasé. Llegaron a mis compañeros, y dijeron que eran remendones, y dijo uno de los diablos: «Deben entender los remendones en el mundo que no se hizo el infierno sino para ellos, según se vienen por acá.» Preguntó otro diablo cuántos eran; respondieron que ciento, y replicó un verdugo mal barbado entrecano: «¿Ciento, y sastres? No pueden ser tan pocos; la menor partida que habemos recibido ha sido de mil y ochocientos. En verdad que estamos por no recibirles.» Afligiéronse ellos; mas al fin entraron. Ved cuáles son los malos, que es para ellos amenaza el no dejarlos entrar en el infierno. Entró el primero[595] un negro, chiquito, rubio, de mal pelo; dió un salto en viéndose allá, y dijo: «Ahora acá estamos tod

In [11]:
' '.join([tokenizer.decode(x).replace(' ', '') for x in inputs['input_ids'][100]])

'[CLS] comenzó ##se ##me a hacer ás ##pera la morada y desa ##pac ##ibles los za ##gua ##nes . fuí entrando poco a poco entre unos sastre ##s que se me llegaron , que iban med ##rosos de los diablos . en la primera entrada hallamos siete demonios escribiendo los que íbamos entrando . pregun ##tár ##on ##me mi nombre ; dí ##je ##le y pasé . llegaron a mis compañeros , y dijeron que eran rem ##endo ##nes , y dijo uno de los diablos : [UNK] deben entender los rem ##endo ##nes en el mundo que no se hizo el infierno sino para ellos , según se vienen por acá . [UNK] preguntó otro diablo cuántos eran ; respondieron que ciento , y replic ##ó un verdu ##go mal barba ##do entre ##cano : [UNK] ¿ ciento , y sastre ##s ? no pueden ser tan pocos ; la menor partida que hab ##emos recibido ha sido de mil y ocho ##cientos . en verdad que estamos por no recibir ##les . [UNK] aflig ##i ##éro ##ns ##e ellos ; mas al fin entraron . ve ##d cuáles son los malos , que es para ellos amenaza el no dejarlos entr

Then, the `labels` of the inputs are defined as a copy of the `input_ids`, and some percentage of the inputs are masked:

In [12]:
inputs['labels'] = inputs.input_ids.detach().clone()
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < MASK_PROB) * (inputs.input_ids != CLS_TOKEN) * (inputs.input_ids != SEP_TOKEN) * (inputs.input_ids != PAD_TOKEN)
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [13]:
mask = mask_arr.bool()
indices = mask.nonzero(as_tuple=False)
inputs.input_ids[indices[:, 0], indices[:, 1]] = MASK_TOKEN 

inputs.input_ids

tensor([[    4,  1098,  1000,  ...,     1,     1,     1],
        [    4,  1413,     0,  ...,     1,     1,     1],
        [    4,  1032,     0,  ...,     1,     1,     1],
        ...,
        [    4,  1057,  9447,  ...,     1,     1,     1],
        [    4,  1032,  3876,  ...,     1,     1,     1],
        [    4,  1265, 30957,  ...,     1,     1,     1]])

## 4. Train with Optimizer

For the processing, and efficiency for training the model, the `inputs` object will be converted to a Dataset object, and there will be defined an optimizer for the training. Also, if there's available GPU, the model will be moved to the GPU:

In [14]:
class OldSpanishDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return self.encodings.input_ids.shape[0]

dataset = OldSpanishDataset(inputs)

In [15]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

model.train()
optim = Adam(model.parameters(), lr=LR)

In [ ]:
%%capture training_output
%%time

for epoch in range(EPOCHS):
    for step, batch in enumerate(dataloader):
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optim.step()
        
        if epoch % 1 == 0 and step % 100 == 0:
            print(f"Epoch {epoch} | step {step:03d} Loss: {loss.item()} ")

    print(f"Epoch {epoch} | step {step:03d} Loss: {loss.item()} [end]")

print("Training completed")

In [ ]:
training_output.show()

And save the model into a file:


In [ ]:
torch.save(model, MODEL_SAVE_PATH)

In [ ]:
del model
torch.cuda.empty_cache()

In [ ]:
model = torch.load(MODEL_SAVE_PATH)
model.to(device)